In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import pmdarima as pm
from pmdarima.utils import diff
from sklearn.metrics import mean_squared_error
import pickle
import sklearn
import statsmodels.tsa.statespace as sm
from sklearn.preprocessing import StandardScaler

In [20]:
pd.set_option('display.max_rows', 500)

In [11]:
df = pd.read_csv('./../data/full_dataset_unscaled.csv')
df['zip'] = df['zip'].map(lambda x: str(x))
df['zip'] = df['zip'].map(lambda x: '0' + x if len(x)<5 else x)
df['datetime']=df['datetime'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df = df.set_index('datetime')

In [16]:
df = df[df['year']>=2015]

In [17]:
x = df.drop(columns = ['zip','year', 'month', 'City', 'State', 'Metro', 'CountyName', 'zri'])

In [19]:
vif_df = pd.DataFrame()
vif_df['feature'] = x.columns
vif_df['vif'] = [variance_inflation_factor(x.values, i)
                          for i in range(len(x.columns))]

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,feature,vif
0,percent_white,431.588578
1,percent_black,258.565427
2,percent_asian,66.153976
3,percent_hispanic,266.995060
4,percent_native_am,2.048765
...,...,...
84,total_firms,152.192208
85,job_creation_rate,2.662574
86,job_destruction_rate,1.704371
87,startup_firms,153.491651


In [21]:
vif_df

,feature,vif
0,percent_white,431.588578
1,percent_black,258.565427
2,percent_asian,66.153976
3,percent_hispanic,266.995060
4,percent_native_am,2.048765
5,percent_other_race,2.259900
6,percent_0_17,inf
7,percent_18_39,inf
8,percent_40_64,inf
9,percent_65+,inf


In [33]:
x2 = x.drop(columns = ['percent_buildings_less_10_units', 'percent_rental_units_occupied', 'percent_0_17', 'percent_18_39', 'percent_40_64', 'percent_65+', 'percent_white', 'percent_commute_less_30', 'bs_total_systems'])

In [34]:
vif_df2 = pd.DataFrame()
vif_df2['feature'] = x2.columns
vif_df2['vif'] = [variance_inflation_factor(x2.values, i)
                          for i in range(len(x2.columns))]

In [35]:
vif_df2

,feature,vif
0,percent_black,5.915780
1,percent_asian,4.018663
2,percent_hispanic,13.076649
3,percent_native_am,1.508699
4,percent_other_race,1.337726
5,percent_rental_units_vacant,6.256295
6,percent_graduate_deg,51.805437
7,percent_bachelors,49.806873
8,percent_associates,83.590163
9,percent_highschool,61.314896
